In [2]:
import smbus2
import time
from IPython.display import clear_output

In [3]:
addr_switch = 0x70
addr_switch = 0x7c
addr_charger = 0x6a
addr_gauge = 0x55
addr_rtc = 0x51
addr_eeprom = 0x50
addr_eepromsn = 0x58

In [4]:
c.close()

NameError: name 'c' is not defined

In [5]:
def read_2(c, addr, reg):
    d = c.read_i2c_block_data(addr, reg, 2)
    #print([bin(x) for x in d])

    d = d[0]| d[1] << 8
    #print(bin(d))
    return d

def crop(number, start=0, end=16):
    # Vytvoření masky s jedničkami na pozicích od start do end
    mask = ((1 << (end - start + 1)) - 1) << start
    
    # Použití masky pro vybrání příslušných bitů
    result = (number & mask) >> start
    
    return result

def comp(data, length=16):

    if data & (1 << (length-1)):
        data = data ^ ((1 << length) - 1)
        data = -(data + 1)

    return data



In [6]:
bin(crop(0b00101100, 2, 6))

'0b1011'

In [8]:
def con():
    return smbus2.SMBus(18)

c = con()

# I2C switch, switch internal I2C bus into USB (FT260) IO

In [9]:
c=con()

In [10]:
c.write_byte_data(addr_switch, 0x01, 0x03) # nastavit I2C switch do USB

OSError: [Errno 5] Input/output error

In [11]:
hex(c.read_byte(addr_switch, 0x01))

OSError: [Errno 5] Input/output error

# Nabíječka 

In [12]:
# Vypnout bateriovy modul
c.write_byte_data(addr_charger, 0x18, 0x0a)

In [13]:
print(bin(c.read_byte_data(addr_charger, 0x16)))


charge_current_ma = 800

c.write_byte_data(addr_charger, 0x02, int(charge_current_ma/40)<<5)
c.write_byte_data(addr_charger, 0x14, 0b00100010)
c.write_byte_data(addr_charger, 0x15, 0b00011101)
c.write_byte_data(addr_charger, 0x16, 0b10100000)
c.write_byte_data(addr_charger, 0x17, 0b01010110)
c.write_byte_data(addr_charger, 0x18, 0b00000000)
c.write_byte_data(addr_charger, 0x19, 0b00000001)


# NTC
c.write_byte_data(addr_charger, 0x1a, 0b10111111)

# ADC
c.write_byte_data(addr_charger, 0x26, 0b10001100)

0b10100001


In [14]:

while True:
    # Zapnout interni ADC prevodnik
    #print(bin(c.read_byte_data(addr_charger, 0x26)))
    #c.write_byte_data(addr_charger, 0x26, 0b10001100)
    
    clear_output()
    print("IBUS ADC", comp(crop(read_2(c, addr_charger, 0x28), 1, 16), 15) * 2, " mA")
    #print(" ")

    print("IBAT ADC", comp(crop(read_2(c, addr_charger, 0x2A), 2, 16), 14) * 4, " mA")
    #print(" ")

    print("VBUS ADC", crop(read_2(c, addr_charger, 0x2C), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VPMID ADC", crop(read_2(c, addr_charger, 0x2E), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VBAT ADC", crop(read_2(c, addr_charger, 0x30), 1, 13) * 1.99, " mV")
    #print(" ")

    print("VSYS ADC", crop(read_2(c, addr_charger, 0x32), 1, 13) * 1.99, " mV")
    #print(" ")

    print("TS ADC", crop(read_2(c, addr_charger, 0x34), 0, 12) * 0.0961, " %")
    #print(" ")

    print("TDIE ADC", crop(read_2(c, addr_charger, 0x36), 0, 12) * 0.5 - 40, " C")
    #print(" ")

    time.sleep(1)
    break


IBUS ADC 246  mA
IBAT ADC 264  mA
VBUS ADC 5045.87  mV
VPMID ADC 5041.900000000001  mV
VBAT ADC 4190.94  mV
VSYS ADC 4220.79  mV
TS ADC 0.0  %
TDIE ADC -22.0  C


In [15]:
# Nastavit timery
c.write_byte_data(addr_charger, 0x15, 0b00001001)

In [17]:
d = c.read_byte_data(addr_charger, 0x1d)
print("Charger status 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1e)
print("Charger status 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1f)
print("Fault reg 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x20)
print("Charger flag 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x21)
print("Charger flag 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x22)
print("Fault flag 0: 0b{0:08b}".format(d))

Charger status 0: 0b00000000
Charger status 1: 0b00010100
Fault reg 0: 0b00000000
Charger flag 0: 0b00000000
Charger flag 1: 0b00000000
Fault flag 0: 0b00000000


In [22]:
voltage = c.read_i2c_block_data(0x55,0x08,2)
voltage = (voltage[1] << 8) | voltage[0]
print(f"Voltage {voltage} mV")

current = c.read_i2c_block_data(0x55,0x0a,2)
current = (current[1] << 8) | current[0]
print(f"Current {current} mA (avg)")

current = c.read_i2c_block_data(0x55,0x10,2)
current = (current[1] << 8) | current[0]
print(f"Current {current} mA")


d = c.read_word_data(0x55,0x04)
print(f"RemainingCapacity {d} mAh")


d = c.read_word_data(0x55,0x06)
print(f"Full Capacity {d} mAh")

d = c.read_word_data(0x55,0x0c)
print(f"Temp {d/10-273} C")


d = c.read_byte(0x55,0x02)
print(f"State: {d} %")


#d = c.read_ (0x55,0x04,2)
#d = (d[1] << 8) | d[0]
#print(f"CRemainingCapacity {d} mAh")

OSError: [Errno 5] Input/output error

In [75]:
import struct 
bus = c
address = 0x55

e=bus.read_i2c_block_data(address,0x0a,2)
(full_cap,)=struct.unpack('H', bytearray(e)[0:2])
print("Full Available Capacity is",full_cap, 'mA')

#Trying to write the nominal capacity
bus.write_byte_data(address,0x00,0x00)
bus.write_byte_data(address,0x01,0x80)
bus.write_byte_data(address,0x00,0x00)
bus.write_byte_data(address,0x01,0x80)

bus.write_byte_data(address,0x00,0x13)
bus.write_byte_data(address,0x01,0x00)

bus.write_byte_data(address,0x61,0x00)
bus.write_byte_data(address,0x3e,0x52)
bus.write_byte_data(address,0x3f,0x00)

bus.write_byte_data(address,0x4a,0x04) #writing new capacity
bus.write_byte_data(address,0x4b,0xb0) #writing new capacity

time.sleep(1)

#bus.write_byte_data(address,0x60,0x1f) #trying to write on BlockDataChecksum() 

time.sleep(1)

bus.write_byte_data(address,0x00,0x42)
bus.write_byte_data(address,0x01,0x00)

bus.write_byte_data(address,0x00,0x20)
bus.write_byte_data(address,0x01,0x00)

f=bus.read_i2c_block_data(address,0x3c,2) #address for design capacity
(des_cap,)=struct.unpack('H', bytearray(f)[0:2])
print(des_cap)


Full Available Capacity is 302 mA


OSError: [Errno 5] Input/output error

EEPROM + Serial number...

In [20]:
c.read_i2c_block_data(addr_eepromsn, 0b00001000, 16)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255]


# RTC

In [40]:
import datetime

In [41]:
def get_time():
    r01 = c.read_byte_data(addr_rtc, 0x01)
    r02 = c.read_byte_data(addr_rtc, 0x02)
    r03 = c.read_byte_data(addr_rtc, 0x03)
    r04 = c.read_byte_data(addr_rtc, 0x04)
    r06 = c.read_byte_data(addr_rtc, 0x06)
    r07 = c.read_byte_data(addr_rtc, 0x07)

    sec = ((r01 >> 4) & 0b111) * 10 + (r01 & 0b1111)
    minu= ((r02 >> 4) & 0b111) * 10 + (r02 & 0b1111)
    hour= ((r03 >> 4) & 0b11) * 10 + (r03 & 0b1111)
    day = ((r04 >> 4) & 0b11) * 10 + (r04 & 0b1111)
    mon = ((r06 >> 4) & 0b1) * 10 + (r06 & 0b1111)
    year= ((r07 >> 4) & 0b1111) * 10 + (r07 & 0b1111)+2000
        
    return datetime.datetime(year, mon, day, hour, minu, sec)
    
def set_time(time):
    c.write_byte_data(addr_rtc, 0x01, (time.second % 10) | int(time.second/10)<<4 )
    c.write_byte_data(addr_rtc, 0x02, (time.minute % 10) | int(time.minute/10)<<4 )
    c.write_byte_data(addr_rtc, 0x03, (time.hour % 10) | int(time.hour/10)<<4 )
    c.write_byte_data(addr_rtc, 0x04, (time.day % 10) | int(time.day/10)<<4 )
    c.write_byte_data(addr_rtc, 0x06, (time.month % 10) | int(time.month/10)<<4 )
    c.write_byte_data(addr_rtc, 0x07, ((time.year-2000) % 10) | int((time.year-2000)/10)<<4 )

In [44]:
actual_time = datetime.datetime.utcnow()
detector_time = get_time()
print("Actual time", actual_time)
print("Detector time", detector_time)
print("Difference", detector_time-actual_time)

Actual time 2023-11-24 01:32:12.897508
Detector time 2023-11-24 01:32:12
Difference -1 day, 23:59:59.102492


In [43]:
actual_time = datetime.datetime.utcnow()
set_time(actual_time)

In [38]:
actual_time.second

2

In [39]:

time = actual_time

In [ ]:
bin( (time.second % 10)<<4)